In [61]:
import os, sys
print(os.listdir('../../'))
print(os.getcwd())
sys.path.append("../")
sys.path.append("../src/network")
sys.path.append("../src")
print(os.getcwd())
import torch
import glob
from torch import optim
import numpy as np
import string
import os, sys
import torch.utils.data as D
import cv2
import numpy as np
from torchvision import transforms
# from src.network.gen_model.gen_model import GenModel_FC
# from torchvision.transforms import v2
from random import choices

import random
import matplotlib.pyplot as plt

['.DS_Store', 'task2_ideas.ipynb', 'non-augmented', 'handwriting-recognition', 'Bachelor-s-thesis', 'ws-siamese', 'Thesis', 'htr_self_supervised', 'Paper-ext', 'lost_cities']
/Users/lisakoopmans/Documents/GitHub/handwriting-recognition/src_semi
/Users/lisakoopmans/Documents/GitHub/handwriting-recognition/src_semi


In [62]:
def read_sia_oov(file):
    """Get data paths and labels (with max_word_len) of images in folder."""
    unique_words = {}
    lens = {}

    with open(file) as data_file:
        lines = data_file.read().splitlines()

    for line in lines:
        line_split = line.split(' ')
                
        if len(line_split) > 2:
            img_path, gt_label = line_split[0], ''.join(line_split[1:])
        else:
            img_path, gt_label = line_split[0], line_split[1]
        
        if gt_label in unique_words.keys():
            unique_words[gt_label] += 1
        else:
            unique_words[gt_label] = 1
            
        
        if len(gt_label) in lens.keys():
                lens[len(gt_label)] += 1
        else:
            lens[len(gt_label)] = 1
                
            
    print(f"number of unique words: {len(unique_words.keys())}")

    lens = dict(sorted(lens.items()))

    print(f"Number of words per word length")
    for key in lens.keys():
        print(key, lens[key])

    return unique_words, lens

In [63]:
def read_htr(folder):
    """Get data paths and labels (with max_word_len) of images in folder."""
    partitions = ['train', 'valid', 'test']
    dataset = []
    wid_dict = {}
    unique_words = {}
    wids = []

    for partition in partitions:
        lens = {}
        print(partition)

        text_file = os.path.join(folder, f"ground_truth_{partition}_filtered.txt")

        with open(text_file, encoding='utf-8') as data_file:
            lines = data_file.read().splitlines()

        for line in lines:
            line_split = line.split(' ')
            wid = None

            if not set(string.digits).isdisjoint(set(line_split[-1])):
                wid = line_split[-1]
                line_split = line_split[:-1]
                
            if len(line_split) > 2:
                img_path, gt_label = line_split[0], ''.join(line_split[1:])
            else:
                img_path, gt_label = line_split[0], line_split[1]

            if gt_label in unique_words.keys():
                unique_words[gt_label] += 1
            else:
                unique_words[gt_label] = 1

            if len(gt_label) in lens.keys():
                lens[len(gt_label)] += 1
            else:
                lens[len(gt_label)] = 1
            
            img_path = img_path.replace("/", "\\")

            if wid is not None:
                wids.append(wid)
                if wid not in wid_dict.keys():
                    wid_dict[wid] = []
                
                wid_dict[wid].append((img_path, gt_label))

            # img_path = os.path.join(folder, partition, img_path)
            dataset.append((img_path, gt_label, wid))
    print(f"number of samples: {len(dataset)}")
    print(f"number of wids: {len(wid_dict.keys())}")

    lens = dict(sorted(lens.items()))

    print(f"Number of words per word length")
    for key in lens.keys():
        print(key, lens[key])

    return dataset, wid_dict, wids, unique_words

## Get unique words siamese data sets

* All oov words
* All unique words

In [64]:
sia_oov_folder = "../../ws-siamese/data/iam/words/supervised_htr/"

sia_train_file = sia_oov_folder + "ground_truth_train_filtered.txt"
sia_oov_valid_file = sia_oov_folder + "ground_truth_valid_oov_filtered.txt"
sia_oov_test_file = sia_oov_folder + "ground_truth_test_oov_filtered.txt"

sia_valid_file = sia_oov_folder + "ground_truth_valid_filtered.txt"
sia_test_file = sia_oov_folder + "ground_truth_test_filtered.txt"

In [65]:
sia_train_words, sia_train_lens = read_sia_oov(sia_train_file)
sia_oov_valid_words, sia_oov_valid_lens = read_sia_oov(sia_oov_valid_file)
sia_oov_test_words, sia_oov_test_lens = read_sia_oov(sia_oov_test_file)

sia_valid_words, sia_valid_lens = read_sia_oov(sia_valid_file)
sia_test_words, sia_test_lens = read_sia_oov(sia_test_file)

print(sia_oov_valid_words.keys())
print(sia_oov_test_words.keys())

sia_oov_words = set(list(sia_oov_valid_words.keys()) + list(sia_oov_test_words.keys()))
sia_words = set(list(sia_oov_valid_words.keys()) + list(sia_oov_test_words.keys()) + list(sia_train_words.keys()) + list(sia_valid_words.keys()) + list(sia_test_words.keys()))
print("total number of oov words:", len(sia_oov_words))
print("total number of train words:", len(sia_train_words.keys()))
print("total number of words:", len(sia_words))

for words in sia_oov_test_words.keys():
    if len(words) == 2:
        print(words)

count = 0
sample_count = 0
for word in sia_valid_words.keys():
    if word not in set(list(sia_train_words.keys())):
        count += 1
        sample_count += sia_valid_words[word]
        sia_oov_words.add(word)
print(count, sample_count, sum(sia_valid_words.values()), len(sia_valid_words.keys()))

print("total number of oov words:", len(sia_oov_words))
print("total number of train words:", len(sia_train_words.keys()))

count = 0
sample_count = 0
for word in sia_test_words.keys():
    if word not in set(list(sia_train_words.keys())):
        count += 1
        sample_count += sia_test_words[word]
        sia_oov_words.add(word)
print(count, sample_count, sum(sia_test_words.values()), len(sia_test_words.keys()))

print("total number of oov words:", len(sia_oov_words))
print("total number of train words:", len(sia_train_words.keys()))

number of unique words: 3161
Number of words per word length
2 6415
3 7706
4 5655
5 3798
6 2706
7 2440
number of unique words: 224
Number of words per word length
2 160
3 357
4 380
5 260
6 250
7 277
number of unique words: 220
Number of words per word length
2 4
3 241
4 404
5 184
6 231
7 129
number of unique words: 1386
Number of words per word length
2 1533
3 1774
4 1306
5 963
6 662
7 570
number of unique words: 1545
Number of words per word length
2 1870
3 2296
4 1739
5 1135
6 780
7 723
dict_keys(['new', 'too', 'loud', 'back', 'writing', 'special', 'cannot', 'wonder', 'stage', 'At', 'man', 'best', 'Duke', 'express', 'years', 'goes', 'Council', 'against', 'ister', 'bundle', 'force', 'sewers', 'mass', 'among', 'Mauro', 'show', 'Richard', 'left', 'avert', 'minimum', 'each', 'odds', 'foreign', 'passed', 'Congo', 'Senator', 'Russia', 'Macleod', 'Tories', 'nothing', 'likely', 'three', 'want', 'social', 'Nuclear', 'capable', 'cinema', 'young', 'women', 'moved', 'mother', 'if', 'she', 'doubt

## Get all samples self-HTR dataset

* Get all samples, wids, gt_labels
* Get all unique words

In [66]:
htr_folder = "../data/iam/words/"
# htr_train_data, htr_train_wids, htr_train_words = read_htr(htr_folder + "ground_truth_train_filtered.txt")
# htr_valid_data, htr_valid_wids, htr_valid_words = read_htr(htr_folder + "ground_truth_valid_filtered.txt")
# htr_test_data, htr_test_wids, htr_test_words = read_htr(htr_folder + "ground_truth_test_filtered.txt")

htr_dataset, htr_wids_dict, htr_wids, htr_words = read_htr(htr_folder)
print("number of uniqe words:", len(list(htr_words.keys())))

train
valid
test
number of samples: 31556
number of wids: 499
Number of words per word length
2 1028
3 1288
4 984
5 663
6 495
7 434
number of uniqe words: 4442


In [67]:
htr_oov_words = []

for word in list(htr_words.keys()):
    if word not in sia_words:
        htr_oov_words.append(word)

print("Number of oov words:", len(htr_oov_words))

htr_oov_samples = []
htr_wids_oov_samples = []

for tup in htr_dataset:
    if tup[1] in htr_oov_words:
        htr_oov_samples.append(tup)
        htr_wids_oov_samples.append(tup[2])

print("Number of oov samples in self-HTR of which labels are not in dataset super-HTR:", len(htr_oov_samples))

full_oov_samples = []
full_wids_oov_samples = []

for tup in htr_dataset:
    if tup[1] in sia_oov_words:
        full_oov_samples.append(tup)
        full_wids_oov_samples.append(tup[2])

print("Number of samples in HTR of which words are in oov super-HTR:", len(full_oov_samples))

htr_iv_samples = []
htr_wids_iv_samples = []

for tup in htr_dataset:
    if tup not in htr_oov_samples:
        if tup not in full_oov_samples:
            htr_iv_samples.append(tup)
            htr_wids_iv_samples.append(tup[2])

print("Number of samples in self-HTR of which super-HTR has seen the word:", len(htr_iv_samples))
print("total:", len(htr_iv_samples) + len(full_oov_samples) + len(htr_oov_samples))

Number of oov words: 1772
Number of oov samples in self-HTR of which labels are not in dataset super-HTR: 2302
Number of samples in HTR of which words are in oov super-HTR: 2356
Number of samples in self-HTR of which super-HTR has seen the word: 26898
total: 31556


In [68]:
os.makedirs('../data/iam/words/semisup/', exist_ok=True)
def write_to_txt(file, data):
    with open(file, 'w') as f:
        for tup in data:
            f.write(f"{tup[0]} {tup[1]} {tup[2]}\n")

### Split oov super-htr in self-htr 50/50

In [69]:
from sklearn.model_selection import train_test_split

X_valid, X_test, = train_test_split(full_oov_samples, test_size=0.5, random_state=42, shuffle=True)
print(len(X_valid), len(X_test))
print(X_valid[0], X_test[0])

write_to_txt("../data/iam/words/semisup/ground_truth_valid_oov_super-in-self_filtered.txt", X_valid)
write_to_txt("../data/iam/words/semisup/ground_truth_test_oov_super-in-self_filtered.txt", X_test)

1178 1178
('..\\data\\iam\\words\\valid\\c02\\c02-017\\c02-017-00-04.png', 'novel', '139') ('..\\data\\iam\\words\\train\\a02\\a02-124\\a02-124-04-04.png', 'End', '037')


### 80/10/10 for oov not in self-htr and not in super-htr

In [70]:
X_train, X_test, = train_test_split(htr_oov_samples, test_size=0.2, random_state=42, shuffle=True)
X_valid, X_test, = train_test_split(X_test, test_size=0.5, random_state=42, shuffle=True)

print(len(X_valid), len(X_test), len(X_train))
print(X_valid[0], X_test[0], X_train[0])

write_to_txt("../data/iam/words/semisup/ground_truth_train_oov_super-self_filtered.txt", X_train)
write_to_txt("../data/iam/words/semisup/ground_truth_valid_oov_super-self_filtered.txt", X_valid)
write_to_txt("../data/iam/words/semisup/ground_truth_test_oov_super-self_filtered.txt", X_test)

230 231 1841
('..\\data\\iam\\words\\train\\r03\\r03-110\\r03-110-06-05.png', 'flats', '668') ('..\\data\\iam\\words\\train\\f07\\f07-084b\\f07-084b-08-06.png', 'garlic', '291') ('..\\data\\iam\\words\\valid\\m01\\m01-115\\m01-115-04-05.png', 'finding', '515')


### 70/15/15 for iv 

In [71]:
X_train, X_test, = train_test_split(htr_iv_samples, test_size=0.3, random_state=42, shuffle=True)
X_valid, X_test, = train_test_split(X_test, test_size=0.5, random_state=42, shuffle=True)

print(len(X_valid), len(X_test), len(X_train))
print(X_valid[0], X_test[0], X_train[0])

write_to_txt("../data/iam/words/semisup/ground_truth_train_iv_super-self_filtered.txt", X_train)
write_to_txt("../data/iam/words/semisup/ground_truth_valid_iv_super-self_filtered.txt", X_valid)
write_to_txt("../data/iam/words/semisup/ground_truth_test_iv_super-self_filtered.txt", X_test)

4035 4035 18828
('..\\data\\iam\\words\\train\\b01\\b01-038\\b01-038-06-04.png', 'gain', '089') ('..\\data\\iam\\words\\valid\\a02\\a02-057\\a02-057-03-05.png', 'day', '030') ('..\\data\\iam\\words\\train\\n02\\n02-049\\n02-049-03-00.png', 'look', '561')
